#### Using SimpleElastix register all non class 1 CT scans and attempt to normalize them into a **healthy** CT scan.

In [33]:
import sys
import os
import csv
from collections import Counter
import argparse
from configparser import ConfigParser
import pathlib
from glob import glob
from random import shuffle
import SimpleITK as sitk # pip install SimpleITK
from tqdm import tqdm # pip install tqdm
import h5py
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [77]:
# get file paths and globals from config file, located in utils/ dir
config = ConfigParser()
config.read('../utils/lung_template_config.ini') #local just for now (need if - else for AWS)
LUNA_PATH = config.get('local', 'LUNA_PATH')
TOY_LUNA_PATH = config.get('local', 'TOY_LUNA_PATH')
UID_CSV = config.get('local', 'CSV_PATH')
IMG_SAVE = config.get('local', 'IMG_SAVE')


In [53]:
# create list of CT scan seriesuids where there is no nodule class of 1
unique_ct = Counter({})
with open(UID_CSV) as doc:
    lines = csv.reader(doc)
    next(lines)
    for row in lines:
        if row[0] in unique_ct:
            unique_ct[row[0]] += int(row[1])
        else:
            unique_ct[row[0]] = int(row[1])
print("Unique CT Scans: " + str(len(unique_ct)))

# remove any UIDs that have a positive nodule
class0_ct = {k:v for k, v in unique_ct.items() if v == 0}
print("Unique CT Scans with no class 1 nodules: " + str(len(class0_ct)))


Unique CT Scans: 76
Unique CT Scans with no class 1 nodules: 28


In [73]:
mhd_one = '1.3.6.1.4.1.14519.5.2.1.6279.6001.221945191226273284587353530424.mhd'
mhd_two = '1.3.6.1.4.1.14519.5.2.1.6279.6001.249314567767437206995861966896.mhd'


In [106]:
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(sitk.ReadImage(TOY_LUNA_PATH + mhd_one)[1])
elastixImageFilter.SetMovingImage(sitk.ReadImage(TOY_LUNA_PATH + mhd_two)[1])

parameterMapVector = sitk.VectorOfParameterMap()
parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
elastixImageFilter.SetParameterMap(parameterMapVector)

elastixImageFilter.Execute() #execute register procedure



<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x182541a5a0> >

In [107]:
result = elastixImageFilter.GetResultImage()
print(result)

Image (0x7feabdf0cbe0)
  RTTI typeinfo:   itk::Image<float, 2u>
  Reference Count: 3
  Modified Time: 171175
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 0
  UpdateMTime: 0
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [512, 682]
  BufferedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [512, 682]
  RequestedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [512, 682]
  Spacing: [0.537109, 0.5]
  Origin: [-268.231, -405]
  Direction: 
1 0
0 1

  IndexToPointMatrix: 
0.537109 0
0 0.5

  PointToIndexMatrix: 
1.86182 0
0 2

  Inverse Direction: 
1 0
0 1

  PixelContainer: 
    ImportImageContainer (0x7feabdcabb40)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, float>
      Reference Count: 1
      Modified Time: 171176
      Debug: Off
      Object Name: 
      Observers: 

In [108]:
print(len(result))

349184


In [110]:
#save only 2d slice.
sitk.WriteImage(result, IMG_SAVE+'result1.tif')

In [ ]:
# Concatenate the ND images into one (N+1)D image
population = ['image1.hdr', ..., 'imageN.hdr']
vectorOfImages = sitk.VectorOfImage()

for filename in population
  vectorOfImages.push_back(sitk.ReadImage(filename))

image = sitk.JoinSeries(vectorOfImages)

# Register
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(image)
elastixImageFilter.SetMovingImage(image)
elastixImageFilter.SetParameterMap(sitk.GetDefaultParameterMap('groupwise'))
elastixImageFilter.Execute()

In [70]:
def load_itk(filename):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(filename)

    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)

    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itkimage.GetOrigin())))

    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))

    return ct_scan, origin, spacing
t = load_itk(TOY_LUNA_PATH + mhd)
t

(array([[[-1002,  -982,  -968, ...,  -972,  -985,  -976],
         [-1000, -1008,  -996, ..., -1001,  -986,  -980],
         [ -975, -1017, -1024, ...,  -991,  -995, -1011],
         ...,
         [ -921,  -925,  -918, ..., -1016, -1008,  -998],
         [ -951,  -916,  -898, ..., -1024, -1011, -1015],
         [ -938,  -893,  -876, ...,  -995,  -996, -1019]],
 
        [[-1021, -1012, -1007, ...,  -995, -1015,  -988],
         [-1022, -1003,  -996, ...,  -994, -1001,  -987],
         [-1019, -1003, -1000, ..., -1009,  -999,  -995],
         ...,
         [ -910,  -922,  -924, ...,  -999, -1023, -1021],
         [ -934,  -930,  -922, ..., -1009, -1023, -1023],
         [ -932,  -927,  -910, ..., -1010,  -993,  -994]],
 
        [[-1009, -1007,  -997, ...,  -975, -1000, -1020],
         [ -988,  -988,  -993, ..., -1006, -1014, -1018],
         [ -992,  -974,  -989, ..., -1024, -1021, -1017],
         ...,
         [ -907,  -882,  -876, ...,  -967,  -980,  -987],
         [ -903,  -890, 